In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import urllib.request


In [ ]:
#df1 = pd.read_csv("small-2oq-c1r.csv", error_bad_lines=False, low_memory=False)
#print(df1)

#df2 = df1.set_index('categories', drop=False)
#df3 = df2.loc["Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops"]
#df3.to_csv('tops_listings_small.csv', index=False)

In [ ]:
df3 = pd.read_csv("tops_listings_small.csv")
df3.shape


In [ ]:
def getImageUrls(prod):
    url_str = prod.imageUrlStr
    img_urls = url_str.split(';')
    return img_urls

In [ ]:


def getImagesFromUrls(urls):
    img=[]
    for url in urls:
        req = urllib.request.urlopen(url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img.append(cv2.imdecode(arr, -1)) # 'Load it as it is'
        print(np.shape(img))
        
    return img


In [ ]:
urls = getImageUrls(df3.iloc[0])

print("total images in this listing are :", len(urls))
urls[0]

image = getImagesFromUrls(urls)
print(len(image))
for i in range (len(image)):
    print(image[i].shape)


In [ ]:
#cv2.imshow('image', image[1])
#cv2.waitKey(2)

In [ ]:
i=0
df3.iloc[i]
#df3.iloc[53].description

In [ ]:
def areImagesSameForProducts(prod1, prod2):
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True
    return False

In [ ]:
#areImagesSameForProducts(df3.iloc[13], df3.iloc[14])



In [ ]:
from scipy import spatial
from gensim import models

model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
s1_afv = avg_feature_vector(df3.iloc[298].description, model=model, num_features=300, index2word_set=index2word_set)
s2_afv = avg_feature_vector(df3.iloc[299].description, model=model, num_features=300, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

In [ ]:
def isDuplicate(prod1, prod2):
    if(prod1.productId == prod2.productId):
        return True
    if(prod1.productUrl == prod2.productUrl):
        return True
    
    #if(prod1.title == prod2.title):
    #    if(prod1.description == prod2.description):
    #        return True
        
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True

    return False


In [ ]:
#df3.shape[0]
count = 0
for i in range(0, (int)(df3.shape[0]/1000.0)):
    for j in range(i+1, (int)(df3.shape[0]/1000.0)):
        if(isDuplicate(df3.iloc[i], df3.iloc[j])):
            count +=1
            print(count, i, j)
            #print("duplicate products:", i,j, df3.iloc[i].productUrl, df3.iloc[j].productUrl)